# Laboratório 01 de IoT com IA usando o InfluxDB - Guia para a aula experimental

Nesse roteiro vamos implementar o método de aprendizado não supervisionado kmeans no dispositivo, nesse caso um NodeMCU 1.0, baseado em um ESP8266.
Para seguir esse roteiro, você precisará de acesso a um banco de dados InfluxDB, e, caso deseje trabalhar com o kit disponibilizado para vocês fora do laboratório, será necessário baixar a Arduino IDE e adicionar o suporte a placa de desenvolvimento **NodeMCU**. Um guia de instalação da placa pode ser visto na referência https://www.filipeflop.com/blog/programar-nodemcu-com-ide-arduino/

Vamos iniciar nosso notebook instalando e posteriormente chamando todas as bibliotecas que usaremos. Nessa etapa, nada precisa ser modificado.

In [ ]:
!pip install arrow influxdb_client

In [ ]:
import numpy as np
import pandas as pd
import arrow
import requests
import re
import influxdb_client, os, time
from influxdb_client import InfluxDBClient, Point, WritePrecision
from influxdb_client.client.write_api import SYNCHRONOUS
import matplotlib.pyplot as plt

O próximo passo é definir os endereços que serão usados para consultar e enviar os dados, o token de acesso, a organização e o bucket com os dados.

In [ ]:
token = ''
user = ''
url = 'http://64.227.106.209:8086'
bucket = ''

Vamos agora conectar com o banco de dados, usando as np.infomções definidas anteriormente.

In [ ]:
db_client = influxdb_client.InfluxDBClient(url=url, token=token, org=user)

## Usando o Kit de Hardware
Esse é o momento de você baixar o código dos dispositivos disponível no GitHub: https://github.com/lfgomez/influxdb/tree/main/kmeans_ESP8266 Você deve baixar o código **kmeans-termistor**. Depois de baixar o código, abra o código do Termometro na Arduino IDE e mude os parâmetros de rede Wifi, canal de publicação (minha sugestão é usar *temperatura*) credenciais do dispositivo.

![term](https://raw.githubusercontent.com/KonkerLabs/arduino_examples/master/Termometro_MQTT/term.jpg "Termômetro")

Com o dispositivo montado, o próximo passo é compilar e gravar o Firmware. Lembre-se de mudar a board na Arduino IDE para **NodeMCU v1.0**.

**Ao iniciar o treinamento, o LED da placa começará a piscar. Após isso, aguarde 10 segundos registrando a temperatura ambiente. Após isso, segure o termistor com seus dedos fazendo a temperatura subir. Mantenha o termômetro aquecido com seus dedos por ao menos 10 segundos. Por último, deixe o dispositivo capturando novamente dados de temperatura ambiente até que o LED da placa pare de piscar.**

**Quando o LED se apagar ou acender continuamente, o treinamento está terminado. A placa então começará a enviar dados para a plataforma, em dois canais: "temperatura" e "cluster". O LED também está configurado para mudar de estado conforme o cluster do dado adquirido: apagado para cluster 0 e aceso para cluster 1.**

## Vamos baixar os dados e ver como eles se comportam
Para iniciar esse trabalho, vamos primeiro escrever uma função que nos permita baixar os dados dos útimos "d" dias, do bucket que definimos anteriormete.

In [ ]:
def get_data(days):
    query_api = db_client.query_api()
    stop = arrow.now().to('UTC').isoformat()[:-13]+'Z'
    start = arrow.now().shift(days=-1*days).to('UTC').isoformat()[:-13]+'Z'
    query = """from(bucket: \""""+bucket+"""\")
    |> range(start: """+start+""", stop: """+stop+""")
    |> pivot(rowKey:["_time"], columnKey: ["_field"], valueColumn: "_value")
    |> keep(columns: ["_time", "_measurement","temperatura", "grupo"])"""
    df = query_api.query_data_frame(query, org=user)
    return df

Agora vamos usar a funçar que definimos acima e baixar o último dia de dados. Vamos aproveitar para deixar o timestamp no horário do Brasil e organizar o Dataframe pelo tempo.

In [ ]:
df = get_data(1)
df['_time'] = df['_time'].dt.tz_convert('America/Sao_Paulo')
df.index = pd.to_datetime(df['_time'])
df = df.drop(columns=['_time'])

Caso tudo tenha funcionado como esperado, você deve estar vendo seus dados na sequência, já no formato tabular do Pandas.

In [ ]:
df

Ótimo! Agora os dados estão em um formato mais fácil de ler. Vamos agora ver os clusters.

In [ ]:
plt.figure(figsize=(15,4))
df['grupo'].plot()
plt.xticks(rotation=45);

Vamos agora plotar os clusters juntamente com a temperatura

In [ ]:
fig, ax1 = plt.subplots(figsize=(15,4))
ax1.set_xlabel('timestamp')
ax1 = df['temperatura'].plot()
ax2 = ax1.twinx()
ax2.fill_between(df.index,df['grupo'], 0,color = 'red', alpha=0.3)
ax1.tick_params(labelrotation=45);

### Perguntas com relação ao kmeans
#### 1) Como funciona o método de kmeans? Quais os parâmetros do método?
#### 2) Ele é um método supervisionado ou não-supervisionado? O que isso significa?
#### 3) O que você espera como resultado dele de uma forma geral? Ele funciona em dimensões superiores a 3?


### Perguntas com relação ao experimento
#### 1) O que ocorre se você tiver um vetor com todas as temperaturas idênticas?
#### 2) Como você inicializaria os centroids? Consegue apontar como foi feito no código? O que ocorre se forem usadas as duas primeiras entradas do vetor como centroids?
#### 3) O que ocorre se você reduzir o máximo de iterações do kmeans para 2? E o que ocorre no limite supeiror, digamos, com o máximo de 100000?

### Perguntas com relação a aprendizado no dispositivo
#### 1) Quais as vantagens do aprendizado no dispositivo? (Latência, disponibilidade, privacidade...)
#### 2) Quais são as maiores desvantagens ou dificuldades no aprendizado de máquina em dispositivos?